In [1]:
from sklearn.metrics import roc_auc_score

In [2]:
import numpy as np
import scipy.stats as stats

In [3]:
import xgboost as xgb
import catboost as cb

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [5]:
from sklearn.model_selection import RandomizedSearchCV

In [6]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [7]:
labels = pd.read_csv('labels_training_set.csv')

In [8]:
features_1 = pd.read_csv('features_07.csv')

In [9]:
features_2 = pd.read_csv('features_06.csv')

In [10]:
repeated = ['busco_productos', 'clickeo_ad', 'hizo_checkout', 'entro_desde_motor',
       'hizo_conversion', 'filtro_busquedas', 'busco_marca', 'visito_sitio',
       'hizo_lead','primer_visita','ultima_visita']

In [11]:
processed = features_1.set_index('person').join(features_2.set_index('person').drop(repeated,axis=1),lsuffix='1')

In [12]:
cat_features = []
for i,col in enumerate(processed.columns):
    if processed[col].dtype.name=='object':
        cat_features.append(i)

In [13]:
predict_features = kaggle.set_index('person').join(processed)

In [14]:
training_features = labels.set_index('person').join(processed)
training_labels = training_features['label']
training_features = training_features.drop(['label'],axis=1)

## Dropeando las unknown (score: 0.86514)

In [65]:
xtrain,xtest,ytrain,ytest = train_test_split(training_features.drop(['Unknown','Unknown.1'],axis=1),training_labels)

In [83]:
cb_classifier = cb.CatBoostClassifier(iterations=400,
                                 depth=8,
                                 l2_leaf_reg=50,
                                 random_strength=0.1,
                                 bagging_temperature=0,
                                 border_count=128)

In [84]:
cat_features2 = []
for i,col in enumerate(xtrain.columns):
    if xtrain[col].dtype.name=='object':
        cat_features2.append(i)

In [ ]:
cb_classifier.fit(xtrain,ytrain,cat_features=cat_features2)

In [86]:
roc_auc_score(ytest,cb_classifier.predict_proba(xtest)[:,1])

0.8533207805546046

In [ ]:
cb_classifier.fit(training_features.drop(['Unknown','Unknown.1'],axis=1),training_labels,cat_features=cat_features2)

In [89]:
proba_loco=cb_classifier.predict_proba(predict_features.drop(['Unknown','Unknown.1'],axis=1))[:,1]

In [90]:
submit = kaggle.set_index('person')
submit['label'] = proba_loco

In [91]:
submit.to_csv('submit.csv')